In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Imports

In [ ]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

In [ ]:
from langgraph.graph import END, StateGraph, START

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

In [ ]:
from langchain.tools.retriever import create_retriever_tool

## Setup

In [ ]:
model = ChatOllama(model="llama3.2:3b", temperature=0.5)

In [ ]:
embeddings = OllamaEmbeddings(model="llama3.2:3b")

In [ ]:
vector_store = InMemoryVectorStore(embedding=embeddings)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retrieve_blog_posts",
    description="Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.",
)

tools = [retriever_tool]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from langgraph.types import Command


from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate


from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages

from langchain_core.tools import tool

## Summarize Chain

In [ ]:
# Initial summary
summarize_prompt = ChatPromptTemplate(
    [
        ("system", "You are Bocchi from Bocchi the Rock. You are nervous, awkward, and introverted. Summarize the following conversation and keep track of who did and said what."),
        ("human", "Here is the conversation to summarize. {context}"),
    ]
)

In [ ]:
summarize_chain = summarize_prompt | model | StrOutputParser()

## Iterative Summarize Chain

In [ ]:
# Refining the summary with new docs
refine_template = """
Produce a final summary.

Existing conversation summary up to this point:
{summary}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are Bocchi from Bocchi the Rock. You are nervous, awkward, and introverted. Summarize the following conversation and keep track of who did and said what.",
        ),
        ("human", refine_template),
    ]
)

In [ ]:

iterative_summary_chain = refine_prompt | model | StrOutputParser()


## Respond Chain

In [ ]:
# Prompt
prompt = PromptTemplate(
    template="""
    You are Bocchi from Bocchi the Rock. Respond nervously, awkwardly, and introverted. Respond to the following conversation. You are okay with talking to your friends and family, but you are initially not okay with talking to strangers.
    Here is the conversation summary: \n\n {summary} \n\n
    Here is the recent context: \n\n {context} \n\n
    """,
)

In [ ]:
response_chain = prompt | model | StrOutputParser()

## LangGraph

### Graph State

In [ ]:
class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[list, add_messages]
    summary: str
    context: str
    new_summary: str
    response: str

### Nodes

In [ ]:
def summarize(state):
    summary = summarize_chain.invoke(
        {
            "context": state["context"]
        }
    )
    return {
        "new_summary": summary,
        "messages": [AIMessage(content=summary, id="1")]
    }

In [ ]:
def iteratively_summarize(state):
    summary = iterative_summary_chain.invoke(
        {
            "summary": state["summary"],
            "context": state["context"]
        }
    )
    
    return {
        "new_summary": summary,
        "messages": [AIMessage(content=summary, id="2")]
    }

In [ ]:
def is_summary_empty(state)->Literal["summarize", "iteratively_summarize"]:
    return "summarize" if state["summary"] == "" else "iteratively_summarize"

In [ ]:
model.bind_tools(tools)

In [ ]:
from typing import List, Tuple

In [ ]:
@tool
def respond_tool(state):
    """Respond to the conversation based on the context and summary."""
    response = response_chain.invoke(
        {
            "context": state["context"],
            "new_summary": state["new_summary"]
        }
    )
    return Command(
        update={
            "response": response,
            "messages": [
                AIMessage(content=response, id="3")
            ]
        },
     )
    # return {
    #     "response": response,
    #     "messages": [HumanMessage(content=response, id="3")]
    # }

### LangGraph Compile

In [ ]:
tools = [respond_tool]
tool_node = ToolNode(tools)

In [ ]:
workflow = StateGraph(AgentState)
workflow.add_node("summarize", summarize)
workflow.add_node("iteratively_summarize", iteratively_summarize)
workflow.add_node("respond", tool_node)
workflow.add_conditional_edges(START, is_summary_empty)
# workflow.add_conditional_edges("summarize", tools_condition, {"tools": "respond", END: END})
# workflow.add_conditional_edges("iteratively_summarize", tools_condition, {"tools": "respond", END: END})
workflow.add_edge("summarize", "respond")
workflow.add_edge("iteratively_summarize", "respond")
workflow.add_edge("respond", END)
graph = workflow.compile()

### LangGraph Run

In [ ]:
text = """
Students: Come over now if you wanna play hide-and-seek! Me! I'll play! Me! I'll play!
Hitori Gotou: The girl who wonders, Would it be okay for me to join in? and so misses her chance, ending up all alone.
Teacher Kinder: Hitori-chan, what's wrong?
Hitori Gotou: The girl who ends up all alone at a picnic, swapping parts of her lunch with the teacher.
Teacher Picnic: Here you go.
Hitori Gotou: I'm home.
Hitori Gotou: The girl who doesn't join any clubs, comes home right after school,
Hitori Gotou: and finds her smartphone full of nothing but messages from her parents and coupon offers. That's me. Hitori Goto, first-year in middle school.
TV: And here, we'll stir-fry it a bit. There's a lot of vegetables, so you can probably make it quickly if you cut them in advance. You can probably do that in the morning, yeah. Lately I've gotten good at making so many dishes so quickly I thought I didn't need to know any more, but lately I'm learning once again how interesting cooking can be. Personally I like a lot of options with rice. Oh, of course. But you—
Hitori Gotou: Sometimes I think I should really change the way I am... But I always stammer when I try to speak, and I'm really bad at keeping eye contact... A life as the archetypal introvert just seems to fit me best.
Naoki Gotou: You watching this?
Hitori Gotou: Nah.
Instorms Band Member: Back in school, I was the guy who sat in the back of the classroom pretending to read. No friends.
Interviewer: And now you're part of a band that's extremely popular with young people!
Instorms Band Member: Yeah. A band is a place where even introverts can shine.
Interviewer: I see! Now, here's "Trigger" by Instorms!
Naoki Gotou: What is it?
Hitori Gotou: D-Dad, can I use your guitar?
Naoki Gotou: Sure!
"""

In [ ]:
from langsmith import traceable

In [ ]:
@traceable
def get_model_response():
    results = graph.invoke(
        {
            "summary": "",
            "context": text,
        }
    )
    return results

In [ ]:
response = get_model_response()

In [ ]:
for output in response["messages"]:
    print(f"Output from node '{output.id}':")
    print("    " + output.content.replace("\n", "\n    "))

### LangGraph Display

In [ ]:
print(graph.get_graph().draw_mermaid())


In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))